## Alcholic Anonymous News articles extraction from Hindustan Times. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.hindustantimes.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
max_pages = round([int(s) for s in soup.select_one('#resultStats').text.split() if s.isdigit()][0]/10)

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

55 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.storyText').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 4min 11sndustantimes.com/india/walking-with-the-wolves/story-cba3Rd5C2GRSLdHtvM9S6J.htmlg-11-year-old-co-star-s-genitals/story-1Shtd0zN2W5pLLCW9e3fGO.htmlmll.html


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

55 55 55 55 55 55 55 55


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})

tbl.dropna()
tbl.to_csv('Hindustan_Times.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries
0,"Women alcoholics on the rise in Delhi, and acr...","States like Kerala, Andhra Pradesh, Manipur, M...",None,2017-12-10T08:05Z,When a sexagenarian wobbled up to the podium a...,"[rise, alcoholics, alcohol, think, women, indi...","According to members of Shakti, the Delhi grou..."
1,Benefits of AA Meetings,The fact that many have won the battle over th...,None,2003-06-28T21:20Z,You are among people who have or had problems ...,"[groupp, does, problems, benefits, experiences...",You are among people who have or had problems ...
2,"Narcotics Anonmyous helps kick drug addiction,...","Narcotics Anonymous is a non-profit, internati...",None,2015-12-06T14:10Z,He was only eight when his father handed him h...,"[narcotics, members, life, drug, addiction, ha...",That was till he found solace in Narcotics Ano...
3,The destroyer of all that is… good,"Fire gives us warmth, but it also burns us. We...",None,2012-07-01T23:11Z,"Fire gives us warmth, but it also burns us. We...","[alcohol, disease, good, consumed, close, pers...",Eye-Opener: Have you now reached a stage in yo...
4,How to get rid of your addiction,We all like the occasional drink. Or two. Or t...,None,2011-12-10T19:55Z,We all like the occasional drink. Or two. Or t...,"[later, rehab, addiction, doing, wanted, went,...","We got three men and women, all former addicts..."
